# Hypothesis Testing

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. 

The following data files are available for this project:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. Only look at GDP data from the first quarter of 2000 onward.


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib as plt

In [10]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [11]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Read data into dataframe with tab as a seperator
    df = pd.read_csv('university_towns.txt', sep='\t', header=None)
    
    # Remove every character from " (" to the end
    df.replace(" \(.*", value="", inplace=True, regex=True)
    
    # Initiate unicersity_towns dataframe
    university_towns = pd.DataFrame(columns=['State','RegionName'])
    
    # Add rows with State and RegionName to university_towns
    for i in range(len(df)):
        if '[edit]' in str(df.loc[i].values):
            state = str(df.loc[i].values)[2:-8]
        else:
            region = df.loc[i].values[0]
            university_towns = university_towns.append({'State': state, 'RegionName': region}, ignore_index=True)
    return university_towns
#get_list_of_university_towns()

In [12]:
def get_gdp():
    '''Returns the dataframe with quarter and corresponding 
    GDP starting from Quarter1 of 2000'''

    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.drop(df.columns[[0,1,2,3,6,7]], axis=1)
    df.columns = ['Quarter', 'GDP']
    return df[df['Quarter'] >= '2000q1'].reset_index(drop=True)
#get_gdp()

In [13]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = get_gdp()
    for i in range(len(df)-2):
        if (df['GDP'].loc[i+2] < df['GDP'].loc[i]) and (df['GDP'].loc[i+1] < df['GDP'].loc[i]):
            return df['Quarter'].loc[i]
#get_recession_start()

In [14]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = get_gdp()
    df = df[df['Quarter'] >= get_recession_start()].reset_index(drop=True)
    for i in range(len(df)-3):
        if (df['GDP'].loc[i+2] > df['GDP'].loc[i]) and (df['GDP'].loc[i+1] > df['GDP'].loc[i]):
            return df['Quarter'].loc[i+2]
#get_recession_end()

In [15]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    start = get_recession_start()
    end = get_recession_end()
    df = get_gdp()
    df = df[(df['Quarter']>=start) & (df['Quarter']<=end)]
    return df['Quarter'].loc[df['GDP'].idxmin()]
#get_recession_bottom()

In [16]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Read csv into dataframe df
    df = pd.read_csv('City_Zhvi_AllHomes.csv')

    # Map State code to State Names using dictionary 'states' and set index as State and RegionName
    df.replace({'State': states}, inplace=True)

    # Set index as State and RegionName
    df.set_index(['State','RegionName'],inplace=True)

    # Reduce dataframe to only columns we need
    df = df.drop(df.columns[[0,1,2,3]], axis = 1).drop(df.columns[df.columns < '2000-01'], axis=1)
    #df = df.drop(df.columns[df.columns < '2000-01'], axis=1)

    # Convert columns to Pandas DataTime
    df.columns = pd.to_datetime(df.columns)

    # Resample dataframe based on quarters on axis = columns
    df = df.resample('Q',axis=1).mean()

    # Rename columns as 2005q3
    df = df.rename(columns=lambda x: str(x.to_period('Q')).lower())
    return df
#convert_housing_data_to_quarters()

In [17]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Get the housing data and replace Nan with 0
    df = convert_housing_data_to_quarters()

    # Get the quarter before recession start
    start = pd.to_datetime(get_recession_start())
    start = start - pd.Timedelta(days=90)
    qrt_before_rec_start = str(start.to_period('Q')).lower()

    # Get the recession bottom quarter
    bottom = get_recession_bottom()

    # drop all other columns from dataframe ecxept for qurter before recession and recession bottom and
    # calculate the price ratio and replace NaN with 0
    df = df[[qrt_before_rec_start, bottom]]
    df['Price_Ratio'] = df[qrt_before_rec_start]/df[bottom]

    # Separate data based on university town and non university towns
    university_towns = get_list_of_university_towns().to_records(index=False).tolist()
    ut = df['Price_Ratio'].loc[df.index.isin(university_towns)]#.replace('NaN', value=0)
    non_ut = df['Price_Ratio'].loc[~df.index.isin(university_towns)]#.replace('NaN', value=0)
    
    # Run ttest on university town data and non university town data
    stat, p = ttest_ind(ut, non_ut, nan_policy='omit')

    different = False
    if p<0.01:
        different = True

    better = 'non-university town'
    if ut.mean() < non_ut.mean():
        better = 'university town'

    return (different, p, better)
run_ttest()

(True, 0.0027240637047531249, 'university town')